In [1]:
library(dplyr)
library(tidyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



## DATA WITH ICD

In [2]:
source("/u/project/sriram/ukbiobank/33127/bd_load_data.R")
db<-data


Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last



40006-0.0 Type of cancer: ICD10
40013-0.0 Type of cancer: ICD9
41202-0.0 Diagnoses - main ICD10
41203-0.0 Diagnoses - main ICD9
41204-0.0 Diagnoses - secondary ICD10
41205-0.0 Diagnoses - secondary ICD9
41270-0.0 Diagnoses - ICD10
41271-0.0 Diagnoses - ICD9

In [3]:
ICD_var<- c('40006-0.0','40013-0.0','41202-0.0','41203-0.0','41204-0.0',
           '41205-0.0','41270-0.0','41271-0.0')
ICD<- db %>% select(c('eid',ICD_var))%>% mutate_all(na_if,"") 

In [4]:
dim(ICD)

[1] 502508      9

In [5]:
head(ICD)

eid,40006-0.0,40013-0.0,41202-0.0,41203-0.0,41204-0.0,41205-0.0,41270-0.0,41271-0.0
<int>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1923142,NA,NA,NA,NA,NA,NA,NA,NA
5896937,C449,NA,N950,NA,E039,NA,E039,NA
2847894,NA,NA,D122,NA,E119,NA,D122,NA
5764431,NA,NA,K635,NA,I846,NA,I846,NA
4695056,NA,NA,C711,NA,G939,NA,C711,NA
2456604,NA,NA,I672,NA,B378,NA,B378,NA


In [6]:
sapply(ICD,function(x) sum(!is.na(x))/length(x))

eid  40006-0.0  40013-0.0  41202-0.0  41203-0.0  41204-0.0  41205-0.0 
1.00000000 0.16612273 0.01247343 0.81652829 0.04040334 0.67509572 0.01733903 
 41270-0.0  41271-0.0 
0.81654023 0.04040334

In [7]:
ICD <- ICD %>% select(c('eid','41202-0.0','41204-0.0','41270-0.0'))

In [8]:
head(ICD)

eid,41202-0.0,41204-0.0,41270-0.0
<int>,<chr>,<chr>,<chr>
1923142,NA,NA,NA
5896937,N950,E039,E039
2847894,D122,E119,D122
5764431,K635,I846,I846
4695056,C711,G939,C711
2456604,I672,B378,B378


In [9]:
ICD202<-  ICD %>% select(c('eid','41202-0.0'))

In [10]:
ICD270<-  ICD %>% select(c('eid','41270-0.0'))

In [11]:
ICD204<-  ICD %>% select(c('eid','41204-0.0'))

## ICD CATEGORIES

In [12]:
ICD10<-read.csv('phecode_icd10.csv')

In [13]:
ICD10$ICD10<-gsub("\\.","",ICD10$ICD10)

In [14]:
ICD10<- ICD10 %>% mutate_all(na_if,"") 

In [15]:
head(ICD10)

ICD10,ICD10.String,PheCode,Phenotype,Excl..Phecodes,Excl..Phenotypes
<chr>,<fct>,<dbl>,<fct>,<fct>,<fct>
A00,Cholera,8.0,Intestinal infection,001-009.99,Intestinal infection
A000,"Cholera due to Vibrio cholerae 01, biovar cholerae",8.0,Intestinal infection,001-009.99,Intestinal infection
A001,"Cholera due to Vibrio cholerae 01, biovar eltor",8.0,Intestinal infection,001-009.99,Intestinal infection
A009,"Cholera, unspecified",8.0,Intestinal infection,001-009.99,Intestinal infection
A01,Typhoid and paratyphoid fevers,8.0,Intestinal infection,001-009.99,Intestinal infection
A010,Typhoid fever,8.5,Bacterial enteritis,001-009.99,Intestinal infection


## MATCH

In [16]:
ICD202_withlabel<- merge(ICD202,ICD10[,c('ICD10','Excl..Phenotypes')], by.x = '41202-0.0',by.y = 'ICD10',all.x = TRUE)
ICD202_withlabel <- droplevels(ICD202_withlabel)
ICD202_withlabel<- ICD202_withlabel %>% filter(!is.na(`41202-0.0`))

In [17]:
table_202<-as.data.frame(table(ICD202_withlabel$Excl..Phenotypes))
table_202<-table_202[order(table_202$Freq,decreasing =TRUE),]

In [18]:
head(table_202,20)

,Var1,Freq
,<fct>,<int>
55,Cataract,17618
129,Malignant neoplasm of skin,17489
78,diseases of veins and lymphatics,16127
116,ISCHEMIC HEART DISEASE,15054
36,Benign and malignant neoplasm of colon,14029
47,cancer of female genital organs; breast dysplasia,13350
19,Abdominal hernia,12279
139,noninfective gastrointestinal disorders,11723
130,mononeuritis and nerve lesions,10227


In [19]:
ICD270_withlabel<- merge(ICD270,ICD10[,c('ICD10','Excl..Phenotypes')], by.x = '41270-0.0',by.y = 'ICD10',all.x = TRUE)
ICD270_withlabel <- droplevels(ICD270_withlabel)
ICD270_withlabel<- ICD270_withlabel %>% filter(!is.na(`41270-0.0`))

In [20]:
table_270<-as.data.frame(table(ICD270_withlabel$Excl..Phenotypes))
table_270<-table_270[order(table_270$Freq,decreasing =TRUE),]

In [21]:
head(table_270,20)

,Var1,Freq
,<fct>,<int>
33,bacterial infection,18585
81,Disorders of lipoid metabolism,17723
104,HYPERTENSIVE DISEASE,16588
126,Malignant neoplasm of skin,16467
69,DIABETES,12748
34,Benign and malignant neoplasm of colon,12511
28,Anemias,12323
76,diseases of veins and lymphatics,12305
45,cancer of female genital organs; breast dysplasia,12232


In [22]:
ICD204_withlabel<- merge(ICD204,ICD10[,c('ICD10','Excl..Phenotypes')], by.x = '41204-0.0',by.y = 'ICD10',all.x = TRUE)
ICD204_withlabel <- droplevels(ICD204_withlabel)
ICD204_withlabel<- ICD204_withlabel %>% filter(!is.na(`41204-0.0`))

In [23]:
table_204<-as.data.frame(table(ICD204_withlabel$Excl..Phenotypes))
table_204<-table_204[order(table_204$Freq,decreasing =TRUE),]

In [24]:
head(table_204,20)

,Var1,Freq
,<fct>,<int>
106,HYPERTENSIVE DISEASE,28269
83,Disorders of lipoid metabolism,23800
34,bacterial infection,17690
71,DIABETES,17431
89,DISORDERS OF THYROID GLAND,15738
29,Anemias,11657
198,Substance-related disorders,9939
4,"chronic obstructive pulmonary disease, bronchiectasis, asthma",8655
147,Obesity,8138


In [25]:
c1<-table_202 %>% filter(Freq>10000) %>% select(Var1)
c2<-table_270 %>% filter(Freq>10000) %>% select(Var1)
c3<-table_204 %>% filter(Freq>10000) %>% select(Var1)
var<-unique(c(as.character(c1$Var1),as.character(c2$Var1),as.character(c3$Var1)))

In [26]:
var

[1] "Cataract"                                         
 [2] "Malignant neoplasm of skin"                       
 [3] "diseases of veins and lymphatics"                 
 [4] "ISCHEMIC HEART DISEASE"                           
 [5] "Benign and malignant neoplasm of colon"           
 [6] "cancer of female genital organs; breast dysplasia"
 [7] "Abdominal hernia"                                 
 [8] "noninfective gastrointestinal disorders"          
 [9] "mononeuritis and nerve lesions"                   
[10] "esophageal disorders"                             
[11] "bacterial infection"                              
[12] "Disorders of lipoid metabolism"                   
[13] "HYPERTENSIVE DISEASE"                             
[14] "DIABETES"                                         
[15] "Anemias"                                          
[16] "DISORDERS OF THYROID GLAND"

## TOTAL

In [55]:
ICD_withlabel<- merge(ICD,ICD10[,c('ICD10','Excl..Phenotypes')], by.x = '41202-0.0',by.y = 'ICD10',all.x = TRUE)
ICD_withlabel<- merge(ICD_withlabel,ICD10[,c('ICD10','Excl..Phenotypes')], by.x = '41270-0.0',by.y = 'ICD10',all.x = TRUE)
ICD_withlabel<- merge(ICD_withlabel,ICD10[,c('ICD10','Excl..Phenotypes')], by.x = '41204-0.0',by.y = 'ICD10',all.x = TRUE)

In [28]:
colnames(ICD_withlabel)

[1] "41204-0.0"          "41270-0.0"          "41202-0.0"         
[4] "eid"                "Excl..Phenotypes.x" "Excl..Phenotypes.y"
[7] "Excl..Phenotypes"

In [56]:
ICD_withlabel<-ICD_withlabel %>% filter(Excl..Phenotypes.x %in% var | Excl..Phenotypes.y %in% var |Excl..Phenotypes.x %in% var  )

In [57]:
ICD_withlabel<-ICD_withlabel %>% select(-c('41204-0.0','41270-0.0','41202-0.0'))

In [58]:
colnames(ICD_withlabel)<- c('eid','d1','d2','d3')

In [60]:
ICD_withlabel$d1[!ICD_withlabel$d1 %in% var]<-NA
ICD_withlabel$d2[!ICD_withlabel$d2 %in% var]<-NA
ICD_withlabel$d3[!ICD_withlabel$d3 %in% var]<-NA

In [61]:
ICD_withlabel$d1<-factor(ICD_withlabel$d1)
ICD_withlabel$d2<-factor(ICD_withlabel$d2)
ICD_withlabel$d3<-factor(ICD_withlabel$d3)

In [125]:
same<-function(i)
    {
    r<- ICD_withlabel[i,]
    r<-c(as.character(r[,2]),as.character(r[,3]),as.character(r[,4]))
    r<-r[!is.na(r)]
    return(unique(r))
}

In [126]:
result<-as.data.frame(ICD_withlabel$eid)
result$d1<-NA
result$d2<-NA
result$d3<-NA

In [127]:
for(i in 1:dim(result)[1])
    {
    r<-same(i)
    if(length(r)==1)
        {
        r <- c(r,NA,NA)
    }
    else if(length(r) ==2)
        {
        r<-c(r,NA)
    }
    result[i,2:4]<-r
}

In [111]:
same(3353)

[1] "ISCHEMIC HEART DISEASE" "bacterial infection"

In [71]:
ICD_withlabel[which(ICD_withlabel$d1!=ICD_withlabel$d2 ),]

,eid,d1,d2,d3
,<int>,<fct>,<fct>,<fct>
3353,3793169,ISCHEMIC HEART DISEASE,bacterial infection,bacterial infection
3363,3119762,ISCHEMIC HEART DISEASE,bacterial infection,bacterial infection
3393,4736711,ISCHEMIC HEART DISEASE,bacterial infection,bacterial infection
3395,3735566,Cataract,bacterial infection,bacterial infection
3398,5430291,Benign and malignant neoplasm of colon,bacterial infection,bacterial infection
3407,2277091,Cataract,bacterial infection,bacterial infection
3408,1462425,mononeuritis and nerve lesions,bacterial infection,bacterial infection
3418,2600398,Benign and malignant neoplasm of colon,bacterial infection,bacterial infection
3426,1463473,noninfective gastrointestinal disorders,bacterial infection,bacterial infection


In [66]:
tail(ICD_withlabel,50)

,eid,d1,d2,d3
,<int>,<fct>,<fct>,<fct>
223997,1150532,diseases of veins and lymphatics,diseases of veins and lymphatics,NA
223998,5534278,esophageal disorders,esophageal disorders,NA
223999,4643634,esophageal disorders,esophageal disorders,NA
224000,1764436,noninfective gastrointestinal disorders,noninfective gastrointestinal disorders,NA
224001,5557054,noninfective gastrointestinal disorders,noninfective gastrointestinal disorders,NA
224002,1533559,Cataract,Cataract,NA
224003,2432337,esophageal disorders,esophageal disorders,NA
224004,5192226,diseases of veins and lymphatics,diseases of veins and lymphatics,NA
224005,3620273,noninfective gastrointestinal disorders,noninfective gastrointestinal disorders,NA


In [118]:
head(result)

ICD_withlabel$eid,d1,d2,d3
<int>,<chr>,<chr>,<chr>
4984090,Benign and malignant neoplasm of colon,NA,NA
4304000,HYPERTENSIVE DISEASE,NA,NA
5228869,esophageal disorders,NA,NA
1849766,mononeuritis and nerve lesions,NA,NA
3805396,Anemias,NA,NA
5106857,Cataract,NA,NA


In [122]:
result[!is.na(result$d3),]

,ICD_withlabel$eid,d1,d2,d3
,<int>,<chr>,<chr>,<chr>
3494,2289244,Malignant neoplasm of skin,NA,bacterial infection
3496,2180052,Malignant neoplasm of skin,NA,bacterial infection
5000,1623257,noninfective gastrointestinal disorders,NA,bacterial infection
5008,2481857,diseases of veins and lymphatics,NA,bacterial infection
5024,1207555,ISCHEMIC HEART DISEASE,NA,bacterial infection
5028,4987084,ISCHEMIC HEART DISEASE,NA,bacterial infection
5044,4090434,cancer of female genital organs; breast dysplasia,NA,bacterial infection
5046,5522069,Malignant neoplasm of skin,NA,bacterial infection
5060,3285857,ISCHEMIC HEART DISEASE,NA,bacterial infection


In [124]:
r<-same(3494)
r[!is.na(r)]

[1] "Malignant neoplasm of skin" "bacterial infection"

In [128]:
write.csv(result,'result.csv',row.names=F)